# Hotel Recommender
> UCB Extension Data Analytics Boot Camp Mod. 6 Challenge

In [23]:
import pandas as pd
import numpy as np
import gmaps
import requests
import matplotlib.pyplot as plt


In [24]:
city_data = pd.read_csv('cities.csv')
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tuktoyaktuk,CA,2022-04-10 23:42:59,69.4541,-133.0374,3.20,66,20,16.11
1,1,Zeya,RU,2022-04-10 23:47:20,53.7500,127.2667,30.88,44,100,5.91
2,2,Torbay,CA,2022-04-10 23:47:21,47.6666,-52.7314,35.28,94,100,11.50
3,3,Nikolskoye,RU,2022-04-10 23:47:21,59.7035,30.7861,36.81,92,6,8.34
4,4,Puerto Ayora,EC,2022-04-10 23:47:22,-0.7393,-90.3518,78.55,93,96,5.01


In [25]:
from configparser import ConfigParser
config = ConfigParser()
config.read('/Users/laurenbayson/Documents/Anything Code/my Jypter work/tweets/config.ini')
gmap_api = config.get('google_cloud','gmap_api_key')


## Customer input
- Max and min temp preference


In [4]:
min_temp = float(input('Minimum Temperature(F) you would like for your vacation? '))
max_temp = float(input('Maximum Temperature(F) you would like for your vacation? '))

Minimum Temperature(F) you would like for your vacation? 80
Maximum Temperature(F) you would like for your vacation? 110


## Hotels based on user input

In [5]:
filtered_city = city_data.loc[(city_data['Max Temp']<=max_temp)&
                                 (city_data['Max Temp']>=min_temp)]
filtered_city

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,San Patricio,US,2022-04-10 23:47:22,28.0170,-97.5169,81.54,71,88,26.62
8,8,Vaitape,PF,2022-04-10 23:47:24,-16.5167,-151.7500,81.75,75,9,17.02
14,14,Kapaa,US,2022-04-10 23:47:27,22.0752,-159.3190,80.11,82,100,12.66
16,16,Padilla,CO,2022-04-10 23:47:28,3.2204,-76.3139,80.17,95,100,1.88
48,48,Avarua,CK,2022-04-10 23:47:41,-21.2078,-159.7750,84.25,74,40,3.44
...,...,...,...,...,...,...,...,...,...,...
522,522,Cotonou,BJ,2022-04-10 23:51:27,6.3654,2.4183,84.18,79,20,9.22
527,527,Oistins,BB,2022-04-10 23:51:29,13.0667,-59.5333,80.69,74,75,20.71
535,535,Lagos,NG,2022-04-10 23:50:30,6.5833,3.7500,80.92,89,74,6.78
536,536,Eydhafushi,MV,2022-04-10 23:51:33,5.1033,73.0708,84.61,72,30,13.71


In [6]:
filtered_city.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [7]:
clean_filtered_cities = filtered_city.dropna()
clean_filtered_cities

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,San Patricio,US,2022-04-10 23:47:22,28.0170,-97.5169,81.54,71,88,26.62
8,8,Vaitape,PF,2022-04-10 23:47:24,-16.5167,-151.7500,81.75,75,9,17.02
14,14,Kapaa,US,2022-04-10 23:47:27,22.0752,-159.3190,80.11,82,100,12.66
16,16,Padilla,CO,2022-04-10 23:47:28,3.2204,-76.3139,80.17,95,100,1.88
48,48,Avarua,CK,2022-04-10 23:47:41,-21.2078,-159.7750,84.25,74,40,3.44
...,...,...,...,...,...,...,...,...,...,...
522,522,Cotonou,BJ,2022-04-10 23:51:27,6.3654,2.4183,84.18,79,20,9.22
527,527,Oistins,BB,2022-04-10 23:51:29,13.0667,-59.5333,80.69,74,75,20.71
535,535,Lagos,NG,2022-04-10 23:50:30,6.5833,3.7500,80.92,89,74,6.78
536,536,Eydhafushi,MV,2022-04-10 23:51:33,5.1033,73.0708,84.61,72,30,13.71


### Gather Hotels

In [8]:
hotel_df = clean_filtered_cities[["City", "Country", "Max Temp", "Lat", "Lng"]]
hotel_df['Hotel Name'] = ''
hotel_df.head()

/var/folders/2k/4k9pck9d72570x622d52vdcw0000gn/T/ipykernel_1883/212102660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Max Temp,Lat,Lng,Hotel Name
5,San Patricio,US,81.54,28.0170,-97.5169,
8,Vaitape,PF,81.75,-16.5167,-151.7500,
14,Kapaa,US,80.11,22.0752,-159.3190,
16,Padilla,CO,80.17,3.2204,-76.3139,
48,Avarua,CK,84.25,-21.2078,-159.7750,


In [9]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_api
}

# Iterate through the hotel df
for index,row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotels = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,'Hotel Name']=hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not found...Skip')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip
Hotel not found...Skip


In [36]:
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!='')]
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,San Patricio,US,81.54,28.0170,-97.5169,"Motel 6 Sinton, TX"
8,Vaitape,PF,81.75,-16.5167,-151.7500,Conrad Bora Bora Nui
14,Kapaa,US,80.11,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Padilla,CO,80.17,3.2204,-76.3139,La Granadina
48,Avarua,CK,84.25,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...
499,Catuday,PH,80.67,16.2923,119.8062,Benjie's Kubo & Cottages
522,Cotonou,BJ,84.18,6.3654,2.4183,Novotel Cotonou Orisha
527,Oistins,BB,80.69,13.0667,-59.5333,Butterfly Beach Hotel
536,Eydhafushi,MV,84.61,5.1033,73.0708,Soneva Fushi


## Save to CSV

In [11]:
output_data_file = 'Weather_vacation.csv'
clean_filtered_cities.to_csv(output_data_file,index_label='City_ID')